In [ ]:
#                                             Annotation

In [1]:
import omicverse as ov
print(f'omicverse version:{ov.__version__}')
import scanpy as sc
print(f'scanpy version:{sc.__version__}')
ov.ov_plot_set()


   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

Version: 1.5.8, Tutorials: https://omicverse.readthedocs.io/
omicverse version:1.5.8
scanpy version:1.9.6


In [2]:
adata = sc.read_h5ad("filtered_300adata.h5ad")

In [ ]:
#adata=ov.single.scanpy_lazy(adata)

#quantity control
adata=ov.pp.qc(adata,
              tresh={'mito_perc': 0.05, 'nUMIs': 500, 'detected_genes': 250})
#normalize and high variable genes (HVGs) calculated
adata=ov.pp.preprocess(adata,mode='shiftlog|pearson',n_HVGs=2000,)

#save the whole genes and filter the non-HVGs
adata.raw = adata0
adata = adata[:, adata.var.highly_variable_features]

#scale the adata.X
ov.pp.scale(adata)

#Dimensionality Reduction
ov.pp.pca(adata,layer='scaled',n_pcs=50)

#Neighbourhood graph construction
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50,
               use_rep='scaled|original|X_pca')

#clusters
sc.tl.leiden(adata)

#Dimensionality Reduction for visualization(X_mde=X_umap+GPU)
adata.obsm["X_mde"] = ov.utils.mde(adata.obsm["scaled|original|X_pca"])
adata

In [3]:
adata=ov.single.scanpy_lazy(adata)

filtered out 72998 cells that have less than 200 genes expressed
Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:05)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.71
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 24.1%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 0.3%
    Scrublet finished (0:02:10)
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispers

In [ ]:
"""
## 一步部署
#adata=ov.single.scanpy_lazy(adata)

# 质控
adata=ov.pp.qc(adata,
              tresh={'mito_perc': 0.05, 'nUMIs': 500, 'detected_genes': 250})   ##阈值条件

# normalize and high variable genes (HVGs)
adata=ov.pp.preprocess(adata,mode='shiftlog|pearson',n_HVGs=2000,)  ##选取2000个HVGs

# 保存原始数据，并且提取HVGs
adata.raw = adata
adata = adata[:, adata.var.highly_variable_features]

# scale化
ov.pp.scale(adata)

# 降低维度
ov.pp.pca(adata,layer='scaled',n_pcs=50)

#Neighbourhood graph construction ## 领域可视化
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50,
               use_rep='scaled|original|X_pca')     
use_rep：指定使用哪个嵌入向量或矩阵来计算细胞间的距离。

这里有三种选项：
•'scaled'：使用经过归一化或标准化处理的原始表达矩阵。
•'original'：使用未经PCA变换的原始表达矩阵。
•'X_pca'：使用先前计算好的PCA降维后的数据。



#clusters  聚类
sc.tl.leiden(adata)

#Dimensionality Reduction for visualization(X_mde=X_umap+GPU)   可视化降维
adata.obsm["X_mde"] = ov.utils.mde(adata.obsm["scaled|original|X_pca"])
adata
"""

In [4]:
adata

AnnData object with n_obs × n_vars = 46672 × 197
    obs: 'fov', 'Area', 'AspectRatio', 'Width', 'Height', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD68', 'Max.CD68', 'Mean.Membrane', 'Max.Membrane', 'Mean.CD45', 'Max.CD45', 'Mean.DAPI', 'Max.DAPI', 'cell_id', 'assay_type', 'version', 'Run_Tissue_name', 'Panel', 'slide_ID_numeric', 'cell_ID', 'unassignedTranscripts', 'nCount_RNA', 'nFeature_RNA', 'nCount_negprobes', 'nFeature_negprobes', 'nCount_falsecode', 'nFeature_falsecode', 'Area.um2', 'x_FOV_px', 'y_FOV_px', 'x_slide_mm', 'y_slide_mm', 'sample', 'leiden', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'DisplayName', 'Barcode', 'CodeClass', 'ProbeID', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'log1p', 'neighbors', 'scaled|original|cum_sum_eigenvalues', 'scaled|o

In [30]:
## 注意：需要先下载“pySCSA_2023_v2_plus.db”这个数据库，然后
scsa=ov.single.pySCSA(adata=adata,
                      foldchange=1.5,
                      pvalue=0.01,
                      celltype='normal',
                      target='cellmarker',
                      tissue='All',
                      model_path='temp/pySCSA_2023_v2_plus.db' )

In [12]:
scsa

In [31]:
anno=scsa.cell_anno(clustertype='leiden',
               cluster='all',rank_rep=True)

ranking genes
    finished (0:00:15)
...Auto annotate cell
Version V2.1 [2023/06/27]
DB load: GO_items:47347,Human_GO:3,Mouse_GO:3,
CellMarkers:82887,CancerSEA:1574,PanglaoDB:24223
Ensembl_HGNC:61541,Ensembl_Mouse:55414
Version V2.1 [2023/06/27]
DB load: GO_items:47347,Human_GO:3,Mouse_GO:3,
CellMarkers:82887,CancerSEA:1574,PanglaoDB:24223
Ensembl_HGNC:61541,Ensembl_Mouse:55414
load markers: 70276
Cluster 0 Gene number: 28
Other Gene number: 127
Cluster 1 Gene number: 1
Other Gene number: 151
Cluster 10 Gene number: 3
Other Gene number: 150
Cluster 2 Gene number: 5
Other Gene number: 147
Cluster 3 Gene number: 29
Other Gene number: 139
Cluster 4 Gene number: 47
Other Gene number: 114
Cluster 5 Gene number: 30
Other Gene number: 132
Cluster 6 Gene number: 28
Other Gene number: 127
Cluster 7 Gene number: 5
Other Gene number: 149
Cluster 8 Gene number: 3
Other Gene number: 149
Cluster 9 Gene number: 2
Other Gene number: 151
#Cluster Type Celltype Score Times
['0', '?', 'Mesenchymal stem c